# FizzBuzz

FizzBuzz是一个简单的小游戏。游戏规则如下：从1开始往上数数，当遇到3的倍数的时候，说fizz，当遇到5的倍数，说buzz，当遇到15的倍数，就说fizzbuzz，其他情况下则正常数数。

我们可以写一个简单的小程序来决定要返回正常数值还是fizz, buzz 或者 fizzbuzz。

In [2]:
# One-hot encode the desired outputs: [number, "fizz", "buzz", "fizzbuzz"]
def fizz_buzz_encode(i):#独热编码的一种（两个def）
    if   i % 15 == 0: return 3
    elif i % 5  == 0: return 2
    elif i % 3  == 0: return 1
    else:             return 0
    
def fizz_buzz_decode(i, prediction):
    return [str(i), "fizz", "buzz", "fizzbuzz"][prediction]


In [3]:
import numpy as np
import torch

NUM_DIGITS = 10

# Represent each input by an array of its binary digits.
def binary_encode(i, num_digits):
    return np.array([i >> d & 1 for d in range(num_digits)])

trX = torch.Tensor([binary_encode(i, NUM_DIGITS) for i in range(101, 2 ** NUM_DIGITS)])
#trY是LongTensor，因为X是float输入没有问题，但Y是表示类别的量，只能是0,1,2,3
trY = torch.LongTensor([fizz_buzz_encode(i) for i in range(101, 2 ** NUM_DIGITS)])

In [4]:
trY

tensor([0, 1, 0, 0, 3, 0, 0, 1, 0, 2, 1, 0, 0, 1, 2, 0, 1, 0, 0, 3, 0, 0, 1, 0,
        2, 1, 0, 0, 1, 2, 0, 1, 0, 0, 3, 0, 0, 1, 0, 2, 1, 0, 0, 1, 2, 0, 1, 0,
        0, 3, 0, 0, 1, 0, 2, 1, 0, 0, 1, 2, 0, 1, 0, 0, 3, 0, 0, 1, 0, 2, 1, 0,
        0, 1, 2, 0, 1, 0, 0, 3, 0, 0, 1, 0, 2, 1, 0, 0, 1, 2, 0, 1, 0, 0, 3, 0,
        0, 1, 0, 2, 1, 0, 0, 1, 2, 0, 1, 0, 0, 3, 0, 0, 1, 0, 2, 1, 0, 0, 1, 2,
        0, 1, 0, 0, 3, 0, 0, 1, 0, 2, 1, 0, 0, 1, 2, 0, 1, 0, 0, 3, 0, 0, 1, 0,
        2, 1, 0, 0, 1, 2, 0, 1, 0, 0, 3, 0, 0, 1, 0, 2, 1, 0, 0, 1, 2, 0, 1, 0,
        0, 3, 0, 0, 1, 0, 2, 1, 0, 0, 1, 2, 0, 1, 0, 0, 3, 0, 0, 1, 0, 2, 1, 0,
        0, 1, 2, 0, 1, 0, 0, 3, 0, 0, 1, 0, 2, 1, 0, 0, 1, 2, 0, 1, 0, 0, 3, 0,
        0, 1, 0, 2, 1, 0, 0, 1, 2, 0, 1, 0, 0, 3, 0, 0, 1, 0, 2, 1, 0, 0, 1, 2,
        0, 1, 0, 0, 3, 0, 0, 1, 0, 2, 1, 0, 0, 1, 2, 0, 1, 0, 0, 3, 0, 0, 1, 0,
        2, 1, 0, 0, 1, 2, 0, 1, 0, 0, 3, 0, 0, 1, 0, 2, 1, 0, 0, 1, 2, 0, 1, 0,
        0, 3, 0, 0, 1, 0, 2, 1, 0, 0, 1,

In [5]:
# Define the model
NUM_HIDDEN = 100#NUM_DIGITS = 10，trX = torch.Tensor([binary_encode(i, NUM_DIGITS) for i in range(101, 2 ** NUM_DIGITS)])
#直接从nn.Sequential继承，不需要super
model = torch.nn.Sequential(
    torch.nn.Linear(NUM_DIGITS, NUM_HIDDEN),
    torch.nn.ReLU(),
    torch.nn.Linear(NUM_HIDDEN, 4)
).cuda(device = torch.device("cuda:3") )

In [6]:
torch.cuda.is_available()

True

In [7]:
loss_fn = torch.nn.CrossEntropyLoss()
#model.parameters()意味着所有参数全部导入
optimizer = torch.optim.SGD(model.parameters(), lr = 0.05)

In [8]:
# 训练过程
#训练数据trX长900多，一个batch设长128，一个epoch训练走完多个bach
#epoch纪元
#batch批量
#pytorch的 crossentropyloss 输入的就是scalar（纯量，标量）不是one hot
import torchsnooper


BATCH_SIZE = 128
def train():
    for epoch in range(10000):
        for start in range(0, len(trX), BATCH_SIZE):
            end = start + BATCH_SIZE#end用来截取数据
            batchX = trX[start:end]
            batchY = trY[start:end]
        
            batchX=batchX.cuda(device = torch.device("cuda:3"))
            batchY=batchY.cuda(device = torch.device("cuda:3"))#把模型搬到GPU上
        
            y_pred = model(batchX)#拿到预测
            loss = loss_fn(y_pred, batchY)
            
            print('Epoch',epoch,loss.item())

        #优化的三个步骤
        #1、zerograd()
        #2、loss.backward()
        #3、优化器进行一步优化
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()#SGD随机梯度下降


train()

Epoch 0 1.410542368888855
Epoch 0 1.358905553817749
Epoch 0 1.3034170866012573
Epoch 0 1.293510913848877
Epoch 0 1.2484028339385986
Epoch 0 1.2204118967056274
Epoch 0 1.1946568489074707
Epoch 0 1.1636961698532104
Epoch 1 1.1907793283462524
Epoch 1 1.188515067100525
Epoch 1 1.1603810787200928
Epoch 1 1.1859623193740845
Epoch 1 1.163419485092163
Epoch 1 1.154405117034912
Epoch 1 1.1547940969467163
Epoch 1 1.123424768447876
Epoch 2 1.1576584577560425
Epoch 2 1.162821888923645
Epoch 2 1.1400889158248901
Epoch 2 1.1676214933395386
Epoch 2 1.150349736213684
Epoch 2 1.1425532102584839
Epoch 2 1.1495853662490845
Epoch 2 1.1175748109817505
Epoch 3 1.1480567455291748
Epoch 3 1.1552132368087769
Epoch 3 1.134456992149353
Epoch 3 1.1616088151931763
Epoch 3 1.1466501951217651
Epoch 3 1.138658881187439
Epoch 3 1.1485099792480469
Epoch 3 1.1160725355148315
Epoch 4 1.144149661064148
Epoch 4 1.1518908739089966
Epoch 4 1.1323609352111816
Epoch 4 1.158864974975586
Epoch 4 1.1452538967132568
Epoch 4 1.1369

Epoch 47 1.1312655210494995
Epoch 47 1.1493840217590332
Epoch 47 1.078532099723816
Epoch 48 1.1355847120285034
Epoch 48 1.1377710103988647
Epoch 48 1.1292591094970703
Epoch 48 1.1468086242675781
Epoch 48 1.14263117313385
Epoch 48 1.1312326192855835
Epoch 48 1.1494179964065552
Epoch 48 1.0779680013656616
Epoch 49 1.135493278503418
Epoch 49 1.137649655342102
Epoch 49 1.1292332410812378
Epoch 49 1.1466857194900513
Epoch 49 1.142601728439331
Epoch 49 1.1311851739883423
Epoch 49 1.1494344472885132
Epoch 49 1.0774348974227905
Epoch 50 1.1354113817214966
Epoch 50 1.137535810470581
Epoch 50 1.1292104721069336
Epoch 50 1.146564245223999
Epoch 50 1.1425777673721313
Epoch 50 1.1311520338058472
Epoch 50 1.1494567394256592
Epoch 50 1.0768656730651855
Epoch 51 1.1353226900100708
Epoch 51 1.13742196559906
Epoch 51 1.129180908203125
Epoch 51 1.1464407444000244
Epoch 51 1.1425526142120361
Epoch 51 1.1311168670654297
Epoch 51 1.1494925022125244
Epoch 51 1.0763237476348877
Epoch 52 1.1352293491363525
Epo

Epoch 98 1.1279937028884888
Epoch 98 1.1411644220352173
Epoch 98 1.1412442922592163
Epoch 98 1.1290682554244995
Epoch 98 1.150482416152954
Epoch 98 1.0550694465637207
Epoch 99 1.1311955451965332
Epoch 99 1.1325182914733887
Epoch 99 1.1279575824737549
Epoch 99 1.1410523653030396
Epoch 99 1.1412142515182495
Epoch 99 1.129015326499939
Epoch 99 1.1504979133605957
Epoch 99 1.0546810626983643
Epoch 100 1.1310992240905762
Epoch 100 1.1324150562286377
Epoch 100 1.127933382987976
Epoch 100 1.1409543752670288
Epoch 100 1.1411842107772827
Epoch 100 1.1289496421813965
Epoch 100 1.1504966020584106
Epoch 100 1.0543060302734375
Epoch 101 1.1310107707977295
Epoch 101 1.1323169469833374
Epoch 101 1.1278905868530273
Epoch 101 1.1408439874649048
Epoch 101 1.1411542892456055
Epoch 101 1.1288974285125732
Epoch 101 1.150518536567688
Epoch 101 1.053902506828308
Epoch 102 1.130915880203247
Epoch 102 1.1322096586227417
Epoch 102 1.127866506576538
Epoch 102 1.1407393217086792
Epoch 102 1.141125202178955
Epoch 1

Epoch 153 1.1386408805847168
Epoch 153 1.124790072441101
Epoch 153 1.150012731552124
Epoch 153 1.0338327884674072
Epoch 154 1.1248797178268433
Epoch 154 1.1263389587402344
Epoch 154 1.1246695518493652
Epoch 154 1.1346570253372192
Epoch 154 1.138567328453064
Epoch 154 1.124679446220398
Epoch 154 1.1499788761138916
Epoch 154 1.033423900604248
Epoch 155 1.1247403621673584
Epoch 155 1.1262078285217285
Epoch 155 1.1245886087417603
Epoch 155 1.134508728981018
Epoch 155 1.1385000944137573
Epoch 155 1.1245492696762085
Epoch 155 1.1499289274215698
Epoch 155 1.0329703092575073
Epoch 156 1.124618649482727
Epoch 156 1.1260731220245361
Epoch 156 1.1245049238204956
Epoch 156 1.1343601942062378
Epoch 156 1.1384445428848267
Epoch 156 1.1244308948516846
Epoch 156 1.1498886346817017
Epoch 156 1.032514214515686
Epoch 157 1.124444603919983
Epoch 157 1.1259642839431763
Epoch 157 1.1243970394134521
Epoch 157 1.1342403888702393
Epoch 157 1.138365626335144
Epoch 157 1.1242892742156982
Epoch 157 1.149815440177

Epoch 206 1.1180806159973145
Epoch 206 1.1242729425430298
Epoch 206 1.1338763236999512
Epoch 206 1.1168514490127563
Epoch 206 1.145914077758789
Epoch 206 1.006857991218567
Epoch 207 1.1141544580459595
Epoch 207 1.1170209646224976
Epoch 207 1.117952585220337
Epoch 207 1.1239838600158691
Epoch 207 1.1337405443191528
Epoch 207 1.1166667938232422
Epoch 207 1.1457737684249878
Epoch 207 1.0063318014144897
Epoch 208 1.113857626914978
Epoch 208 1.1168296337127686
Epoch 208 1.1177735328674316
Epoch 208 1.123731255531311
Epoch 208 1.1336026191711426
Epoch 208 1.1164860725402832
Epoch 208 1.1456904411315918
Epoch 208 1.0058046579360962
Epoch 209 1.1136195659637451
Epoch 209 1.1165765523910522
Epoch 209 1.1176443099975586
Epoch 209 1.1234327554702759
Epoch 209 1.1334984302520752
Epoch 209 1.1162959337234497
Epoch 209 1.1455391645431519
Epoch 209 1.005232810974121
Epoch 210 1.113332986831665
Epoch 210 1.1163722276687622
Epoch 210 1.1174731254577637
Epoch 210 1.1231920719146729
Epoch 210 1.133359313

...

Epoch 9797 0.010059187188744545
Epoch 9797 0.0068256426602602005
Epoch 9797 0.007210584357380867
Epoch 9797 0.010087506845593452
Epoch 9797 0.005585435777902603
Epoch 9797 0.000928843452129513
Epoch 9798 0.006292132660746574
Epoch 9798 0.0081978440284729
Epoch 9798 0.010058626532554626
Epoch 9798 0.006821289658546448
Epoch 9798 0.007213756442070007
Epoch 9798 0.010084014385938644
Epoch 9798 0.0055824946612119675
Epoch 9798 0.0009284195839427412
Epoch 9799 0.006290523335337639
Epoch 9799 0.008199330419301987
Epoch 9799 0.010054182261228561
Epoch 9799 0.006825331598520279
Epoch 9799 0.007207198068499565
Epoch 9799 0.010084880515933037
Epoch 9799 0.0055810268968343735
Epoch 9799 0.0009297617943957448
Epoch 9800 0.006290990859270096
Epoch 9800 0.008198896422982216
Epoch 9800 0.010051509365439415
Epoch 9800 0.00682525709271431
Epoch 9800 0.0072060395032167435
Epoch 9800 0.010082747787237167
Epoch 9800 0.005583534017205238
Epoch 9800 0.0009274306357838213
Epoch 9801 0.0062877945601940155
Epo

Epoch 9839 0.009979547932744026
Epoch 9839 0.00678216852247715
Epoch 9839 0.0071546416729688644
Epoch 9839 0.010016538202762604
Epoch 9839 0.005544828251004219
Epoch 9839 0.0009234746103174984
Epoch 9840 0.00624455139040947
Epoch 9840 0.0081454087048769
Epoch 9840 0.009979058057069778
Epoch 9840 0.0067831892520189285
Epoch 9840 0.007161224260926247
Epoch 9840 0.010014863684773445
Epoch 9840 0.005543403327465057
Epoch 9840 0.000924057443626225
Epoch 9841 0.006243234500288963
Epoch 9841 0.008144021034240723
Epoch 9841 0.009975876659154892
Epoch 9841 0.006776571273803711
Epoch 9841 0.0071633681654930115
Epoch 9841 0.01001535914838314
Epoch 9841 0.0055441707372665405
Epoch 9841 0.0009228918352164328
Epoch 9842 0.006243504583835602
Epoch 9842 0.008144179359078407
Epoch 9842 0.00997447781264782
Epoch 9842 0.006776582449674606
Epoch 9842 0.007156841456890106
Epoch 9842 0.010010793805122375
Epoch 9842 0.0055417250841856
Epoch 9842 0.0009235452744178474
Epoch 9843 0.006241818889975548
Epoch 984

Epoch 9890 0.005495712161064148
Epoch 9890 0.0009178585605695844
Epoch 9891 0.0061900001019239426
Epoch 9891 0.00807936117053032
Epoch 9891 0.009885681793093681
Epoch 9891 0.006724866107106209
Epoch 9891 0.007095050066709518
Epoch 9891 0.00993138924241066
Epoch 9891 0.005494451150298119
Epoch 9891 0.0009173640282824636
Epoch 9892 0.006187429651618004
Epoch 9892 0.00807996280491352
Epoch 9892 0.009882235899567604
Epoch 9892 0.006727777421474457
Epoch 9892 0.0071029700338840485
Epoch 9892 0.009926730766892433
Epoch 9892 0.005497569218277931
Epoch 9892 0.0009166752570308745
Epoch 9893 0.006189985200762749
Epoch 9893 0.008076416328549385
Epoch 9893 0.009882675483822823
Epoch 9893 0.006722515448927879
Epoch 9893 0.007100541144609451
Epoch 9893 0.009925710037350655
Epoch 9893 0.005494415760040283
Epoch 9893 0.0009162161150015891
Epoch 9894 0.00618821382522583
Epoch 9894 0.008074646815657616
Epoch 9894 0.009880173951387405
Epoch 9894 0.006725180894136429
Epoch 9894 0.007096810266375542
Epoch 

Epoch 9939 0.007046302780508995
Epoch 9939 0.009854860603809357
Epoch 9939 0.005454603582620621
Epoch 9939 0.0009119422174990177
Epoch 9940 0.006138032302260399
Epoch 9940 0.00801742635667324
Epoch 9940 0.009796932339668274
Epoch 9940 0.006674168631434441
Epoch 9940 0.007037997245788574
Epoch 9940 0.00985165685415268
Epoch 9940 0.00545121356844902
Epoch 9940 0.0009113594423979521
Epoch 9941 0.0061382707208395
Epoch 9941 0.00801655650138855
Epoch 9941 0.009795214980840683
Epoch 9941 0.006677962839603424
Epoch 9941 0.007042476907372475
Epoch 9941 0.00984656810760498
Epoch 9941 0.005453873425722122
Epoch 9941 0.0009107766090892255
Epoch 9942 0.006135670468211174
Epoch 9942 0.00801478698849678
Epoch 9942 0.009793434292078018
Epoch 9942 0.006674913689494133
Epoch 9942 0.007041638717055321
Epoch 9942 0.009848415851593018
Epoch 9942 0.005450073629617691
Epoch 9942 0.0009113947744481266
Epoch 9943 0.0061344001442193985
Epoch 9943 0.008015157654881477
Epoch 9943 0.0097921472042799
Epoch 9943 0.

Epoch 9980 0.0009074034751392901
Epoch 9981 0.006094392389059067
Epoch 9981 0.0079670250415802
Epoch 9981 0.009723963215947151
Epoch 9981 0.006634039804339409
Epoch 9981 0.0069963689893484116
Epoch 9981 0.00978485494852066
Epoch 9981 0.005413036793470383
Epoch 9981 0.0009064321056939662
Epoch 9982 0.006094355136156082
Epoch 9982 0.007965754717588425
Epoch 9982 0.009724657982587814
Epoch 9982 0.006630841642618179
Epoch 9982 0.006990905851125717
Epoch 9982 0.009783748537302017
Epoch 9982 0.005414463579654694
Epoch 9982 0.0009066087077371776
Epoch 9983 0.0060922130942344666
Epoch 9983 0.007965436205267906
Epoch 9983 0.009721292182803154
Epoch 9983 0.006633790209889412
Epoch 9983 0.006996922194957733
Epoch 9983 0.009780095890164375
Epoch 9983 0.005414977669715881
Epoch 9983 0.0009063967736437917
Epoch 9984 0.006093461066484451
Epoch 9984 0.00796385109424591
Epoch 9984 0.009718777611851692
Epoch 9984 0.006628226488828659
Epoch 9984 0.006991362199187279
Epoch 9984 0.009779142215847969
Epoch 

In [10]:
# Output now
testX = torch.Tensor([binary_encode(i, NUM_DIGITS) for i in range(1, 101)])
testX=testX.cuda(device = torch.device("cuda:3"))
with torch.no_grad():
    testY = model(testX)
predictions = zip(range(1, 101), list(testY.max(1)[1].data.tolist()))

print([fizz_buzz_decode(i, x) for (i, x) in predictions])

['1', '2', 'fizz', '4', 'buzz', 'fizz', '7', '8', 'fizz', 'buzz', '11', '12', '13', '14', 'fizzbuzz', '16', '17', 'fizz', '19', 'buzz', 'fizz', '22', '23', '24', 'buzz', '26', '27', '28', '29', 'fizzbuzz', '31', '32', 'fizz', '34', 'buzz', 'fizz', '37', '38', 'fizz', 'buzz', '41', 'fizz', '43', '44', 'fizzbuzz', '46', '47', 'fizz', '49', 'buzz', 'fizz', '52', '53', '54', 'buzz', '56', '57', '58', '59', '60', '61', '62', 'fizz', '64', 'buzz', 'fizz', '67', '68', 'fizz', 'buzz', '71', 'fizz', '73', '74', 'fizzbuzz', '76', '77', 'fizz', '79', 'buzz', '81', '82', '83', '84', 'buzz', '86', 'fizz', '88', '89', 'fizzbuzz', '91', '92', 'buzz', '94', 'buzz', 'fizz', '97', '98', 'fizz', 'buzz']


In [13]:
testY#预测有四个类型，testY是四维向量的列表，tensor([[a,b,c,d],[a,b,c,d]])，代表四种选择的可能性

tensor([[ 1.0547e+01,  5.2893e-01, -3.7495e+00, -7.1670e+00],
        [ 1.1059e+01, -5.0415e+00, -8.4795e-01, -5.3218e+00],
        [ 2.5383e+00,  9.0368e+00, -3.6945e+00, -8.2778e+00],
        [ 4.6698e+00, -2.6135e+00,  3.9152e+00, -5.3275e+00],
        [-2.3489e+00, -1.9808e+00,  9.9585e+00, -4.6005e+00],
        [-5.7515e-01,  3.0108e+00, -5.8709e-01, -2.5458e+00],
        [ 7.0150e+00, -3.9302e+00,  1.0572e+00, -4.7613e+00],
        [ 1.0782e+01, -1.9331e+00,  9.9684e-01, -8.5934e+00],
        [ 1.2671e-02,  4.7781e+00,  2.9126e+00, -7.0105e+00],
        [ 3.5173e+00, -2.6767e+00,  4.8178e+00, -5.1965e+00],
        [ 9.9686e+00, -2.3083e+00, -6.7156e-01, -7.0210e+00],
        [ 3.6458e+00,  2.1294e+00,  2.3514e+00, -6.7241e+00],
        [ 6.2231e+00, -3.8031e+00,  3.4677e+00, -4.5586e+00],
        [ 8.5054e+00, -6.1735e+00, -2.7882e+00,  6.5499e-01],
        [-2.0687e+00, -3.0947e+00,  8.2376e-01,  4.7290e+00],
        [ 1.0682e+01, -2.2173e+00, -2.9637e+00, -5.5704e+00],
        

In [14]:
testY.max()#不加dim (int) 就返回最大值

tensor(12.9595, device='cuda:3')

In [12]:
testY.max(1)#加(1)，返回两个，一个value,一个indices

torch.return_types.max(
values=tensor([10.5471, 11.0593,  9.0368,  4.6698,  9.9585,  3.0108,  7.0150, 10.7824,
         4.7781,  4.8178,  9.9686,  3.6458,  6.2231,  8.5054,  4.7290, 10.6822,
         9.0176,  7.9355,  3.4212,  6.3513,  9.0934,  7.2378,  8.4084,  2.7604,
         7.6495,  9.4326,  4.4069, 10.5286,  7.2922,  6.0262,  8.8377,  9.1283,
         9.5830,  5.9387,  6.3225,  3.0388,  7.3894,  3.5502,  3.6428,  6.3899,
        12.2635,  9.7827,  8.5000,  6.6595,  4.6930,  3.6233,  5.6757,  6.0663,
         7.9426,  8.2096,  4.4714,  7.9361,  7.9288,  3.2683,  7.0099, 11.8790,
         4.4779,  9.2975,  6.7123,  3.7573, 10.1532,  9.3351,  2.6026,  4.5927,
         8.0642,  2.6622,  4.4977,  8.9006,  4.3340,  4.9057, 11.2953,  2.4452,
         7.1416,  6.7825,  2.6850,  9.6471,  9.5353,  5.2081,  6.8951,  8.8516,
         5.0967,  4.7818,  5.3691,  4.3375, 10.6365, 12.8094,  3.0741,  9.7604,
         7.8307,  2.8807,  7.1181, 12.9595,  3.0189,  8.6465,  6.5639,  5.3719,
         

In [18]:
one,two=testY.max(1)
print(one)


tensor([10.5471, 11.0593,  9.0368,  4.6698,  9.9585,  3.0108,  7.0150, 10.7824,
         4.7781,  4.8178,  9.9686,  3.6458,  6.2231,  8.5054,  4.7290, 10.6822,
         9.0176,  7.9355,  3.4212,  6.3513,  9.0934,  7.2378,  8.4084,  2.7604,
         7.6495,  9.4326,  4.4069, 10.5286,  7.2922,  6.0262,  8.8377,  9.1283,
         9.5830,  5.9387,  6.3225,  3.0388,  7.3894,  3.5502,  3.6428,  6.3899,
        12.2635,  9.7827,  8.5000,  6.6595,  4.6930,  3.6233,  5.6757,  6.0663,
         7.9426,  8.2096,  4.4714,  7.9361,  7.9288,  3.2683,  7.0099, 11.8790,
         4.4779,  9.2975,  6.7123,  3.7573, 10.1532,  9.3351,  2.6026,  4.5927,
         8.0642,  2.6622,  4.4977,  8.9006,  4.3340,  4.9057, 11.2953,  2.4452,
         7.1416,  6.7825,  2.6850,  9.6471,  9.5353,  5.2081,  6.8951,  8.8516,
         5.0967,  4.7818,  5.3691,  4.3375, 10.6365, 12.8094,  3.0741,  9.7604,
         7.8307,  2.8807,  7.1181, 12.9595,  3.0189,  8.6465,  6.5639,  5.3719,
         6.8946,  7.6904,  7.7787,  5.30

In [19]:
print(two)

tensor([0, 0, 1, 0, 2, 1, 0, 0, 1, 2, 0, 0, 0, 0, 3, 0, 0, 1, 0, 2, 1, 0, 0, 0,
        2, 0, 0, 0, 0, 3, 0, 0, 1, 0, 2, 1, 0, 0, 1, 2, 0, 1, 0, 0, 3, 0, 0, 1,
        0, 2, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 1, 0, 0, 1, 2, 0, 1,
        0, 0, 3, 0, 0, 1, 0, 2, 0, 0, 0, 0, 2, 0, 1, 0, 0, 3, 0, 0, 2, 0, 2, 1,
        0, 0, 1, 2], device='cuda:3')


In [15]:
testY.max(1)[1]#第二行，即取出下标，下标就可以译码

tensor([0, 0, 1, 0, 2, 1, 0, 0, 1, 2, 0, 0, 0, 0, 3, 0, 0, 1, 0, 2, 1, 0, 0, 0,
        2, 0, 0, 0, 0, 3, 0, 0, 1, 0, 2, 1, 0, 0, 1, 2, 0, 1, 0, 0, 3, 0, 0, 1,
        0, 2, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 1, 0, 0, 1, 2, 0, 1,
        0, 0, 3, 0, 0, 1, 0, 2, 0, 0, 0, 0, 2, 0, 1, 0, 0, 3, 0, 0, 2, 0, 2, 1,
        0, 0, 1, 2], device='cuda:3')

In [12]:
testY=testY.cpu()
print(np.sum(testY.max(1)[1].numpy() == np.array([fizz_buzz_encode(i) for i in range(1,101)])))
testY.max(1)[1].numpy() == np.array([fizz_buzz_encode(i) for i in range(1,101)])

94


array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True, False, False,  True,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False,  True,  True,  True,  True,
        True,  True,  True,  True,  True, False,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])